In [2]:
import pandas as pd
import glob
import os
import re

In [3]:
path = r'C:\Users\sofch\Python\PR1.20\PR1.20\data\data-train'

folders_path_list = []

for folder in os.listdir(path):
    folders_path_list.append(os.path.join(path, folder))


In [4]:
column_names = ['File_Name', 'Content']
df = pd.DataFrame(columns=column_names)

for folder_path in folders_path_list:
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        with open(file_path, 'r', encoding='latin-1') as file:
            content = file.read()
            df = pd.concat([df, pd.DataFrame({'File_Name': [file_name], 'Content': [content]})], ignore_index=True)

In [5]:
df.head(10)

,File_Name,Content
0,1.txt,Chat Conversation Start\n\nU1: Good girls are ...
1,11.txt,"\nChat Conversation Start\n\nU1: Hi, How are y..."
2,12.txt,\nChat Conversation Start\n\n\nU1: Hello How a...
3,13.txt,\n\n\n\nChat Conversation Start\n\nU1: I want...
4,14.txt,"\n\n\nChat Conversation Start\n\n \nU1: Hi, I ..."
5,15.txt,"\n\n\nChat Conversation Start\n\nU1: Hi, I che..."
6,16.txt,\n\n\n\nChat Conversation Start\n\n\nU1: hello...
7,2.txt,\n\n\nChat Conversation Start\n\n\nU1: hi how ...
8,21.txt,\n\n\nChat Conversation Start\n\nU1: hi how ar...
9,22.txt,\n\n\nChat Conversation Start\n\n\nU1: I want ...


In [7]:
p1 = "Chat Conversation Start"
p2 = "Chat Conversation End"

# Define a function to remove p1 and p2
def remove_start_end(text):
    # Remove p1 only the first time it appears
    text = re.sub(rf'^\s*{re.escape(p1)}\s*', '', text)

    # Remove p2 only the last time it appears
    text = re.sub(rf'\s*{re.escape(p2)}\s*$', '', text)

    return text

# Apply the function to the "Content" column
df['Content'] = df['Content'].apply(remove_start_end)

In [8]:
df.head(10)

,File_Name,Content
0,1.txt,U1: Good girls are in bed now haha omg I miss ...
1,11.txt,"U1: Hi, How are you?\n\nU2: Great Thanks! Kind..."
2,12.txt,U1: Hello How are you \n\n\nU2: Hello hello m...
3,13.txt,"U1: I want to ask you about your study, It ha..."
4,14.txt,"U1: Hi, I checked our courses and ask our frie..."
5,15.txt,"U1: Hi, I checked our program page we need to ..."
6,16.txt,U1: hello hello I am so excited to start my ne...
7,2.txt,"U1: hi how are you, long time we did not talk ..."
8,21.txt,"U1: hi how are you, just to update you we have..."
9,22.txt,U1: I want to book a flight to Dubai but Iâm...


In [10]:
#pd.set_option('display.max_colwidth', None)
#df

In [11]:
content_column.tail(5)

11331    From: psyrobtw@ubvmsd.cc.buffalo.edu (Robert W...
11332    From: sandvik@newton.apple.com (Kent Sandvik)\...
11333    From: sandvik@newton.apple.com (Kent Sandvik)\...
11334    From: cutter@gloster.via.mind.org (cutter)\nSu...
11335    Subject: Re: Albert Sabin\nFrom: rfox@charlie....
Name: Content, dtype: object